In [1]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import psycopg2
from config import db_password,g_key
from config import resolution,lat_max,lat_min,lng_max,lng_min
import json
from shapely.geometry import Point,shape
import os
from datetime import datetime,timedelta
from functions import Rmax_calc,recovery_calc

# Read files

In [2]:
# blocks information
file_path = os.path.join('..','..','Resources','blocks_info.csv')
blocks_df = pd.read_csv(file_path)

In [3]:
blocks_df

,lat,lng,lift,cell,stack_finish,leach_start,leach_end,leach_days,tons_per_block,ounces_per_block,rmax
0,65.012000,-147.351429,1,1,2019-03-27,2019-03-30,2019-07-28,120 days,1994.483436,16.809887,0.706002
1,65.012000,-147.350612,1,1,2019-03-27,2019-03-30,2019-07-28,120 days,1994.483436,16.809887,0.706002
2,65.012265,-147.353061,1,1,2019-03-27,2019-03-30,2019-07-28,120 days,1994.483436,16.809887,0.706002
3,65.012265,-147.352245,1,1,2019-03-27,2019-03-30,2019-07-28,120 days,1994.483436,16.809887,0.706002
4,65.012265,-147.351429,1,1,2019-03-27,2019-03-30,2019-07-28,120 days,1994.483436,16.809887,0.706002
...,...,...,...,...,...,...,...,...,...,...,...
5144,65.021816,-147.362857,4,18,2020-09-27,2020-09-30,2021-01-28,120 days,10303.913043,140.390869,0.738674
5145,65.021816,-147.362041,4,18,2020-09-27,2020-09-30,2021-01-28,120 days,10303.913043,140.390869,0.738674
5146,65.021816,-147.361224,4,18,2020-09-27,2020-09-30,2021-01-28,120 days,10303.913043,140.390869,0.738674
5147,65.022082,-147.363673,4,18,2020-09-27,2020-09-30,2021-01-28,120 days,10303.913043,140.390869,0.738674


# Recovery Calculation

In [4]:
blocks_df['stack_finish'] = pd.to_datetime(blocks_df['stack_finish'])
blocks_df['leach_start'] = pd.to_datetime(blocks_df['leach_start'])
blocks_df['leach_end'] = pd.to_datetime(blocks_df['leach_end'])
blocks_df['leach_days'] = blocks_df['leach_end']-blocks_df['leach_start']

In [5]:
blocks_df.iloc[0]['leach_days'].days

120

In [6]:
project_span = (
    datetime(year=2019,month=1,day=1),
    datetime(year=2050,month=12,day=31)
    )
project_days = (project_span[1]-project_span[0]).days
# Create timeline
datelist = pd.date_range(project_span[0], periods=project_days).tolist()
rec_oz_df = pd.DataFrame(index=datelist)
rec_oz_df['oz']=0
rec_oz_df['oz'] = rec_oz_df['oz'].astype(float)
rec_oz_df.head()

,oz
2019-01-01,0.0
2019-01-02,0.0
2019-01-03,0.0
2019-01-04,0.0
2019-01-05,0.0


In [7]:
blocks_df.iloc[0]

lat                              65.012
lng                         -147.351429
lift                                  1
cell                                  1
stack_finish        2019-03-27 00:00:00
leach_start         2019-03-30 00:00:00
leach_end           2019-07-28 00:00:00
leach_days            120 days 00:00:00
tons_per_block              1994.483436
ounces_per_block              16.809887
rmax                           0.706002
Name: 0, dtype: object

In [8]:
rec_oz_df.describe()

,oz
count,11687.0
mean,0.0
std,0.0
min,0.0
25%,0.0
50%,0.0
75%,0.0
max,0.0


In [9]:
def primary_leach(b,k,i):
    r_cum=0
    # want to add this block into the dataframe as a column.
    # this column stores the ounces change through time.
    # the maximum ounces appears when stacking is finished.
    # also need to keep the ounces left to the end of the project
    # rec_oz_df[str(i)] = 0
    # rec_oz_df[str(i)] = rec_oz_df[str(t)].astype(float)
    # rec_oz_df.at[blocks_df.iloc[i]['stack_finish'],str(i)]=blocks_df.iloc[i]['ounces_per_block']
    
    for n in range(1,b['leach_days'].days+1):
        r_date = b['leach_start'] + timedelta(days = n-1)
        r_day = recovery_calc(b['rmax'],r_cum,k,1)
        oz_day = r_day * b['ounces_per_block']
        rec_oz_df.at[r_date,'oz'] += oz_day
        rec_oz_df.at[r_date,str(i)] = rec_oz_df.at[r_date-timedelta(days = 1),str(i)]-oz_day 
        r_cum += r_day

## Think about create a dataframe store each block's remaining ounces throughout project life span. The ounces should be available at the date when stacking is finished.

In [30]:
for i in range(0,len(blocks_df)):
    primary_leach(blocks_df.iloc[i],0.01,i)

rec_oz_df.describe()

,oz,0,1,2,3,4,5,6,7,8,...,5139,5140,5141,5142,5143,5144,5145,5146,5147,5148
count,16836.000000,16836.000000,16836.000000,16836.000000,16836.000000,16836.000000,16836.000000,16836.000000,16836.000000,16836.000000,...,16836.000000,16836.000000,16836.000000,16836.000000,16836.000000,16836.000000,16836.000000,16836.000000,16836.000000,16836.000000
mean,13.758694,16.786409,16.786409,16.786409,16.786409,16.786409,16.786409,16.786409,16.786409,16.786409,...,140.179511,140.179511,140.179511,140.179511,140.179511,140.179511,140.179511,140.179511,140.179511,140.179511
std,27.894489,0.305535,0.305535,0.305535,0.305535,0.305535,0.305535,0.305535,0.305535,0.305535,...,2.746912,2.746912,2.746912,2.746912,2.746912,2.746912,2.746912,2.746912,2.746912,2.746912
min,0.000000,11.366826,11.366826,11.366826,11.366826,11.366826,11.366826,11.366826,11.366826,11.366826,...,91.661575,91.661575,91.661575,91.661575,91.661575,91.661575,91.661575,91.661575,91.661575,91.661575
25%,0.000000,16.809887,16.809887,16.809887,16.809887,16.809887,16.809887,16.809887,16.809887,16.809887,...,140.390869,140.390869,140.390869,140.390869,140.390869,140.390869,140.390869,140.390869,140.390869,140.390869
50%,0.000000,16.809887,16.809887,16.809887,16.809887,16.809887,16.809887,16.809887,16.809887,16.809887,...,140.390869,140.390869,140.390869,140.390869,140.390869,140.390869,140.390869,140.390869,140.390869,140.390869
75%,16.621023,16.809887,16.809887,16.809887,16.809887,16.809887,16.809887,16.809887,16.809887,16.809887,...,140.390869,140.390869,140.390869,140.390869,140.390869,140.390869,140.390869,140.390869,140.390869,140.390869
max,175.521822,16.809887,16.809887,16.809887,16.809887,16.809887,16.809887,16.809887,16.809887,16.809887,...,140.390869,140.390869,140.390869,140.390869,140.390869,140.390869,140.390869,140.390869,140.390869,140.390869


In [25]:
blocks_df.iloc[0]

lat                              65.012
lng                         -147.351429
lift                                  1
cell                                  1
stack_finish        2019-03-27 00:00:00
leach_start         2019-03-30 00:00:00
leach_end           2019-07-28 00:00:00
leach_days            120 days 00:00:00
tons_per_block              1994.483436
ounces_per_block              16.809887
rmax                           0.706002
Name: 0, dtype: object